In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import libraries here
import warnings
from glob import glob

import pandas as pd
import numpy as np
import seaborn as sns
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_roc_curve

import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter(action="ignore", category=FutureWarning)

## EDA

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

In [ ]:
df.head()

In [ ]:
df.describe()
# as shown in the table below ; id is the unique columne and need to be deleted from the columns

In [ ]:
cols = df.select_dtypes([np.number]).columns

In [ ]:
cols

In [ ]:
cat_cols = list(set( df.columns) - set(cols))

In [ ]:
cat_cols

In [ ]:
len(cols)
# 32 variable is the numerical data except f_27 then we can plot the correlation of the the data to find Multicollinearity  

### box plot

In [ ]:
type(cols)

In [ ]:
cols = cols.drop('id')

In [ ]:
cols

In [ ]:
df[cols].head()

In [ ]:
plt.figure(figsize=(20, 15))
for idx, name in enumerate(cols):
    ax = plt.subplot(8, 4, idx + 1)
    df[cols[idx]].plot(kind='box', vert=False, ax=ax)
    plt.title(name, fontsize=16)
plt.tight_layout()

as shown in the figure above </br>
It seems there are a few ordinal data in the dataset </br>
Consists of f_07, f_08, f_09, f_10, f_11, f_12, f_13, f_14, f_15, f_16, f_17, f_18, f_29, f_30, target

In [ ]:
ordinal_columns = [ 'f_07', 'f_08', 'f_09', 'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16', 'f_17', 'f_18', 'f_29', 'f_30', 'target']

In [ ]:
norminal_columns = cols.drop(ordinal_columns)

In [ ]:
df[ordinal_columns].head()

In [ ]:
df[norminal_columns].head()

There are 3 difference data type in this dataset </br>
Consists of </br>
* Numerical data 
    * 'f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19', 'f_20',
       'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28'
* Caterigal data 
    * 'f_07', 'f_08', 'f_09', 'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15',
       'f_16', 'f_17', 'f_18', 'f_29', 'f_30
* target (binary)
    * 'target'


In [ ]:
df[ordinal_columns] = df[ordinal_columns].astype(str)

In [ ]:
df.describe()

### Multicollinearity

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(df[norminal_columns].corr(), annot=True, ax=ax)
# it seem like there are no any strong multicollinear relationship between the data 

### cat data

In [ ]:
df[cat_cols].head()

In [ ]:
df['f_27_cnt'] = df['f_27'].map(lambda x: len(set(x)))

df['f_27_frequency'] = df['f_27'].map(df['f_27'].value_counts() / len(df))

In [ ]:
drop_columns = ['f_27','f_28']

In [ ]:
df = df.drop(columns = drop_columns)

In [ ]:
df.head()

### wragle

In [ ]:
def wrangle(filename):
    df = pd.read_csv(filename)
    
    # drop unique variable
    df.drop(columns=['id'], inplace=True)
    df[ordinal_columns] = df[ordinal_columns].astype(str)
    
    df['f_27_cnt'] = df['f_27'].map(lambda x: len(set(x)))
    df['f_27_frequency'] = df['f_27'].map(df['f_27'].value_counts() / len(df))
    
    df = df.drop(columns=drop_columns)
    # dropna 
    thresh = len(df) * .5
    df.dropna(axis=1, thresh=thresh, inplace=True)
    return df

In [ ]:
train = wrangle('/kaggle/input/tabular-playground-series-may-2022/train.csv')

In [ ]:
X = train.drop(columns=['target'])
y = train['target'].astype(int)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Build Model
model = make_pipeline(OneHotEncoder(use_cat_names=True), 
                      SimpleImputer(), 
                      GradientBoostingClassifier(n_estimators=200,max_features='auto',max_depth=8,min_samples_leaf=20,verbose=True))

In [ ]:
y_train.mean()

In [ ]:
# Fit model
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
plot_roc_curve(model, X_train, y_train)

In [ ]:
ordinal_columns_test = ordinal_columns.copy()

In [ ]:
ordinal_columns_test.remove('target')

In [ ]:
ordinal_columns_test

In [ ]:
def wrangle_test(filename):
    df = pd.read_csv(filename)
    
    # drop unique variable
    df.drop(columns=['id'], inplace=True)
    df[ordinal_columns_test] = df[ordinal_columns_test].astype(str)
    
    df['f_27_cnt'] = df['f_27'].map(lambda x: len(set(x)))
    df['f_27_frequency'] = df['f_27'].map(df['f_27'].value_counts() / len(df))
    
    df = df.drop(columns=drop_columns)
    # dropna 
    thresh = len(df) * .5
    df.dropna(axis=1, thresh=thresh, inplace=True)
    return df

In [ ]:
test = wrangle_test('/kaggle/input/tabular-playground-series-may-2022/test.csv')

In [ ]:
test.head()

In [ ]:
y_submission = model.predict(test)

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
submission['target'] = y_submission

In [ ]:
submission.to_csv('submission.csv', index=False)